In [ ]:
# Here, the LLM will work as a manager(Coordinator)
# This file contains basic multi-agent system where the coordination among agents is solely managed by a root agent, without using any explicit workflows

In [5]:
# Importing basic libraries
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool, FunctionTool
from google.genai import types
from dotenv import load_dotenv
import os

In [6]:
# Configuration Check
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
if api_key:
    print("API KEY FOUND")
    print("Start building!")
else:
    print("API KEY NOT FOUND")
    print("Configuration Error")

API KEY FOUND
Start building!


In [7]:
# Configuring retry options
retry_config = types.HttpRetryOptions(
    attempts=8, # Max number of retry attempts
    exp_base=7, # Delay multiplier per request
    initial_delay=1, # Delay in first request
    http_status_codes=[429, 500, 503, 504] # Retry only on this specific errors
)

In [8]:
# Building a multi agent system that consists of a research and a summarizer agent
# The job of the research agent is to use the google_search tool and present the latest findings

research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction='''You are a specialized research agent. Your only job is to use the built in google_search tool
    to find 4-5 relevant sources of information on the given topic and present the findings with citations''',
    tools=[google_search],
    output_key="research_findings" # The following key will be used to store the research findings in the session state
)

print("Research agent created")

Research agent created


In [9]:
# The job of this agent is to summarize provided by the research agent
summarizer_agent=Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # Passing the data from the research agent to this agent as the key
    instruction='''Read the provided research findings: {research_findings}
    Create a consise summary of findings with 5-6 key points along with the citations''',
    output_key="final_summary"
)

print("Summarizer agent created")

Summarizer agent created


In [10]:
# The goal of this agent is to act as a coordinator, it orchestrates the workflow by calling the sub-agents as tools
root_agent=Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # Here, the instruction tells the coordinator (Root Agent) how to use its tools which here in this case are the other agents
    instruction='''You are a research coordinator. Your goal is to answer user's query by orchestrating a workflow.
    1. First, you MUST call the 'ResearchAgent' tool to find relevant information on the topic provided by the user.
    2. Next, after receiving the research findings, you MUST call the 'SummarizerAgent' tool to create a concise summary of the findings.
    3. Finally, present the final the final summary clearly as a response to the user.
    ''',
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)]
)

print("Root agent created")

Root agent created


In [12]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest findings in the field of quantum computing and what does it mean for medicinal advancements or creating an entire human"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/arpit/devlopement/agentic-ai-google-kaggle/myvenv/lib/python3.9/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > What are the latest findings in the field of quantum computing and what does it mean for medicinal advancements or creating an entire human


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/arpit/devlopement/agentic-ai-google-kaggle/myvenv/lib/python3.9/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/arpit/devlopement/agentic-ai-google-kaggle/myvenv/lib/python3.9/site-packages/google/adk/agents", which implies app name "agents".


ResearchCoordinator > The latest findings in quantum computing are set to revolutionize medicinal advancements, particularly in drug discovery and personalized medicine. Quantum computers can dramatically reduce the time and cost of developing new drugs by simulating molecular interactions with unprecedented accuracy, potentially shortening timelines from over 10 years to 3-5 years.

In personalized medicine, quantum computing can accelerate genomic analysis, enabling the identification of disease-related genetic markers and the tailoring of treatments to an individual's unique genetic makeup. This can lead to more effective therapies with fewer side effects.

Beyond drug development, quantum computing shows promise in improving diagnostic accuracy, medical imaging, and understanding complex diseases through the modeling of biological systems.

However, significant challenges remain, including the early stage of quantum hardware development, issues with qubit stability and scalability,